In [ ]:
!pip install torch torchvision torchsummary streamlit pyngrok


In [ ]:
!npm install localtunnel

In [ ]:
# prompt: mount g drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%%writefile app.py
import torch
from torchvision import models
import streamlit as st
import os
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from torchsummary import summary
import streamlit as st
from pyngrok import ngrok
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from collections import OrderedDict
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set Web App title
st.title('PNEUMONIA Image Classification')

#Set Header
st.header('Please upload the X-ray Image here: ')

#Upload a file
file=st.file_uploader('', type=['jpeg', 'jpg', 'png'])

##load model
# Assuming the model is saved as 'model.pth' in your Google Drive
model_path = '/content/drive/My Drive/ZikryV3_model.pth'  # Replace with your actual path

# Load the state dictionary
state_dict = torch.load(model_path)

# Determine the number of classes from the saved state dictionary
num_classes = state_dict['fc.weight'].shape[0]  # Get the number of output features in the 'fc' layer

# Create a ResNet model with the correct number of classes
model = models.resnet18(num_classes=num_classes)  # Set num_classes

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Now you can set the model to eval mode
model.eval()

##class Label/classname
with open('/content/drive/My Drive/labels.txt','r') as f:
    class_names = [a[:-1].split(' ')[1] for a in f.readlines()]
    f.close()
print(class_names)

##Display image
if file is not None:
   image = Image.open(file).convert('RGB')
   st.image(image, caption='Uploaded Image.', use_column_width=True)

   # Convert image to (224, 224)
   transform = transforms.Compose([
       transforms.Resize((224, 224)),
       transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ])
   image = transform(image).unsqueeze(0).to(device)

   # Make prediction
   model.eval()
   with torch.no_grad():
       outputs = model(image)
       _, predicted = torch.max(outputs, 1)
       conf_score = torch.nn.functional.softmax(outputs, dim=1)[0][predicted].item()
       class_name = class_names[predicted.item()]

   #write Classification
   st.write("## {}".format(class_name))
   st.write("### score: {}".format(conf_score))



In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501
